In [1]:
from rasa.utils.endpoints import ClientResponseError, EndpointConfig
from rasa.core.agent import Agent
from rasa.core.interpreter import RasaNLUInterpreter
from rasa.model import get_model, get_latest_model

# preq: $ start actions

bot='saya'

endpoint = EndpointConfig("http://localhost:5055/webhook")
bot_locs={'saya': '/pi/ws/sagas-ai/bots/saya'}
bot_loc=get_latest_model(f"{bot_locs[bot]}/models")
print(f'.. load bot model {bot_loc}')
agent = Agent.load(bot_loc, action_endpoint=endpoint)
await agent.handle_text("do you have any restaurants")

.. load bot model /pi/ws/sagas-ai/bots/saya/models/20191201-165323.tar.gz

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.






INFO:tensorflow:Entry Point [tensor2tensor.envs.tic_tac_toe_env:TicTacToeEnv] registered with id [T2TEnv-TicTacToeEnv-v0]


INFO:tensorflow:Restoring parameters from /var/folders/fv/7k1qk5v11dn33sdcngv2wbnm0000gn/T/tmp1w8280xi/nlu/component_6_EmbeddingIntentClassifier.ckpt

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer i

[{'recipient_id': 'default',
  'text': "Found the following objects of type 'restaurant':"},
 {'recipient_id': 'default', 'text': '1: Gong Gan'},
 {'recipient_id': 'default', 'text': '2: Marubi Ramen'},
 {'recipient_id': 'default', 'text': '3: Donath'},
 {'recipient_id': 'default', 'text': '4: Lụa Restaurant'},
 {'recipient_id': 'default', 'text': '5: Berlin Burrito Company'}]

⊕ [Policies](https://rasa.com/docs/rasa/core/policies/#action-selection)

## Action Selection
At every turn, each policy defined in your configuration will predict a next action with a certain confidence level. For more information about how each policy makes its decision, read into the policy’s description below. The bot’s next action is then decided by the policy that predicts with the highest confidence.

In the case that two policies predict with equal confidence (for example, the Memoization and Mapping Policies always predict with confidence of either 0 or 1), the priority of the policies is considered. Rasa policies have default priorities that are set to ensure the expected outcome in the case of a tie. They look like this, where higher numbers have higher priority:

5. FormPolicy
4. FallbackPolicy and TwoStageFallbackPolicy
3. MemoizationPolicy and AugmentedMemoizationPolicy
2. MappingPolicy
1. EmbeddingPolicy, KerasPolicy, and SklearnPolicy

This priority hierarchy ensures that, for example, if there is an intent with a mapped action, but the NLU confidence is not above the nlu_threshold, the bot will still fall back. In general, it is not recommended to have more than one policy per priority level, and some policies on the same priority level, such as the two fallback policies, strictly cannot be used in tandem.

In [5]:
from rasa.core.channels.channel import CollectingOutputChannel
default_channel=CollectingOutputChannel()
policy='MappingPolicy'
# act='action_hello_world'
act='action_query_knowledge_base'
tracker=await agent.execute_action('default', act, default_channel, policy, 1.0)
tracker.current_state()

{'sender_id': 'default',
 'slots': {'attribute': None,
  'bank': None,
  'city': None,
  'cuisine': None,
  'hotel': None,
  'knowledge_base_last_object': None,
  'knowledge_base_last_object_type': 'restaurant',
  'knowledge_base_listed_objects': [6, 5, 4, 2, 1],
  'mention': None,
  'object_type': 'restaurant',
  'person': None,
  'restaurant': None},
 'latest_message': {'intent': {'name': 'query_knowledge_base',
   'confidence': 0.9784830808639526},
  'entities': [{'start': 16,
    'end': 27,
    'value': 'restaurant',
    'entity': 'object_type',
    'confidence': 0.8523547470272897,
    'extractor': 'CRFEntityExtractor',
    'processors': ['EntitySynonymMapper']}],
  'intent_ranking': [{'name': 'query_knowledge_base',
    'confidence': 0.9784830808639526},
   {'name': 'bot_challenge', 'confidence': 0.006442992947995663},
   {'name': 'goodbye', 'confidence': 0.004275038372725248},
   {'name': 'deny', 'confidence': 0.0030048005282878876},
   {'name': 'affirm', 'confidence': 0.0028367

In [6]:
default_channel.latest_output()

{'recipient_id': 'default', 'text': '5: Berlin Burrito Company'}

In [7]:
default_channel.messages

[{'recipient_id': 'default',
  'text': "Found the following objects of type 'restaurant':"},
 {'recipient_id': 'default', 'text': '1: Gong Gan'},
 {'recipient_id': 'default', 'text': '2: Marubi Ramen'},
 {'recipient_id': 'default', 'text': '3: Pfefferberg'},
 {'recipient_id': 'default', 'text': '4: I due forni'},
 {'recipient_id': 'default', 'text': '5: Berlin Burrito Company'}]

In [2]:
await agent.handle_text("do you have any restaurants")

[{'recipient_id': 'default',
  'text': "Found the following objects of type 'restaurant':"},
 {'recipient_id': 'default', 'text': '1: Lụa Restaurant'},
 {'recipient_id': 'default', 'text': '2: I due forni'},
 {'recipient_id': 'default', 'text': '3: Pfefferberg'},
 {'recipient_id': 'default', 'text': '4: Gong Gan'},
 {'recipient_id': 'default', 'text': '5: Marubi Ramen'}]

In [3]:
from rasa.model import get_model, get_latest_model
print(get_model("/pi/ws/sagas-ai/bots/saya/models"))
print(get_latest_model("/pi/ws/sagas-ai/bots/saya/models"))

/var/folders/fv/7k1qk5v11dn33sdcngv2wbnm0000gn/T/tmp6vd0md1b
/pi/ws/sagas-ai/bots/saya/models/20191201-165323.tar.gz


In [4]:
from rasa.core.trackers import DialogueStateTracker
from rasa.core.slots import Slot
from pprint import pprint

tracker = DialogueStateTracker.from_dict("1", [], [Slot("requested_language")])
tracker._set_slot("requested_language", "en")
result = await agent.parse_message_using_nlu_interpreter("In what city is N26 located?", tracker)
pprint(result)

{'entities': [{'confidence': 0.8966057516719946,
               'end': 12,
               'entity': 'attribute',
               'extractor': 'CRFEntityExtractor',
               'start': 8,
               'value': 'city'},
              {'confidence': 0.8079225805462708,
               'end': 19,
               'entity': 'bank',
               'extractor': 'CRFEntityExtractor',
               'start': 16,
               'value': 'N26'}],
 'intent': {'confidence': 0.9932544827461243, 'name': 'query_knowledge_base'},
 'intent_ranking': [{'confidence': 0.9932544827461243,
                     'name': 'query_knowledge_base'},
                    {'confidence': 0.0022818821016699076,
                     'name': 'bot_challenge'},
                    {'confidence': 0.001948829391039908, 'name': 'greet'},
                    {'confidence': 0.0009212908335030079,
                     'name': 'mood_unhappy'},
                    {'confidence': 0.0005631754174828529, 'name': 'deny'},
           

In [5]:
await agent.parse_message_using_nlu_interpreter("In what city is Landesbank Hessen-Thüringen located?", tracker)

{'intent': {'name': 'query_knowledge_base', 'confidence': 0.9964336156845093},
 'entities': [{'start': 8,
   'end': 12,
   'value': 'city',
   'entity': 'attribute',
   'confidence': 0.8964739277115555,
   'extractor': 'CRFEntityExtractor'}],
 'intent_ranking': [{'name': 'query_knowledge_base',
   'confidence': 0.9964336156845093},
  {'name': 'bot_challenge', 'confidence': 0.00170179083943367},
  {'name': 'greet', 'confidence': 0.0008844470139592886},
  {'name': 'mood_unhappy', 'confidence': 0.0003543687053024769},
  {'name': 'affirm', 'confidence': 0.0002737704198807478},
  {'name': 'goodbye', 'confidence': 0.0001746801717672497},
  {'name': 'deny', 'confidence': 0.00017078054952435195},
  {'name': 'mood_great', 'confidence': 6.3904672060743906e-06}],
 'text': 'In what city is Landesbank Hessen-Thüringen located?'}

In [6]:
bank='dekabank deutsche girozentrale'
await agent.parse_message_using_nlu_interpreter(f"In what city is {bank} located?", tracker)

{'intent': {'name': 'query_knowledge_base', 'confidence': 0.9982187151908875},
 'entities': [{'start': 8,
   'end': 12,
   'value': 'city',
   'entity': 'attribute',
   'confidence': 0.8963412926288296,
   'extractor': 'CRFEntityExtractor'}],
 'intent_ranking': [{'name': 'query_knowledge_base',
   'confidence': 0.9982187151908875},
  {'name': 'bot_challenge', 'confidence': 0.0009078445145860314},
  {'name': 'greet', 'confidence': 0.0002828942670021206},
  {'name': 'mood_unhappy', 'confidence': 0.00021179842588026077},
  {'name': 'affirm', 'confidence': 0.00013985982513986528},
  {'name': 'goodbye', 'confidence': 0.00012184304796392098},
  {'name': 'deny', 'confidence': 0.00011239950254093856},
  {'name': 'mood_great', 'confidence': 4.612151769833872e-06}],
 'text': 'In what city is dekabank deutsche girozentrale located?'}

In [7]:
person='Jannik Jung'
await agent.parse_message_using_nlu_interpreter(f"What is the email of {person}?")

{'intent': {'name': 'query_knowledge_base', 'confidence': 0.9926872253417969},
 'entities': [{'start': 12,
   'end': 17,
   'value': 'email',
   'entity': 'attribute',
   'confidence': 0.8052408380922051,
   'extractor': 'CRFEntityExtractor'},
  {'start': 21,
   'end': 32,
   'value': 'Jannik Jung',
   'entity': 'person',
   'confidence': 0.8731735642267661,
   'extractor': 'CRFEntityExtractor'}],
 'intent_ranking': [{'name': 'query_knowledge_base',
   'confidence': 0.9926872253417969},
  {'name': 'bot_challenge', 'confidence': 0.0050545246340334415},
  {'name': 'greet', 'confidence': 0.0010903299553319812},
  {'name': 'mood_unhappy', 'confidence': 0.0004931725561618805},
  {'name': 'deny', 'confidence': 0.00033869585604406893},
  {'name': 'affirm', 'confidence': 0.00023008319840300828},
  {'name': 'goodbye', 'confidence': 9.810854680836201e-05},
  {'name': 'mood_great', 'confidence': 7.864946383051574e-06}],
 'text': 'What is the email of Jannik Jung?'}

In [8]:
person='Finn Hofmann'
await agent.parse_message_using_nlu_interpreter(f"What is the email of {person}?")

{'intent': {'name': 'query_knowledge_base', 'confidence': 0.9757466316223145},
 'entities': [{'start': 12,
   'end': 17,
   'value': 'email',
   'entity': 'attribute',
   'confidence': 0.8051162665335908,
   'extractor': 'CRFEntityExtractor'},
  {'start': 21,
   'end': 33,
   'value': 'Finn Hofmann',
   'entity': 'person',
   'confidence': 0.5439314418415119,
   'extractor': 'CRFEntityExtractor'}],
 'intent_ranking': [{'name': 'query_knowledge_base',
   'confidence': 0.9757466316223145},
  {'name': 'bot_challenge', 'confidence': 0.01303133089095354},
  {'name': 'greet', 'confidence': 0.003075952874496579},
  {'name': 'mood_unhappy', 'confidence': 0.0026953238993883133},
  {'name': 'deny', 'confidence': 0.002415816066786647},
  {'name': 'affirm', 'confidence': 0.001717792241834104},
  {'name': 'goodbye', 'confidence': 0.0011998735135421157},
  {'name': 'mood_great', 'confidence': 0.00011740254558390006}],
 'text': 'What is the email of Finn Hofmann?'}

In [9]:
bank='Wirecard Bank'
q=f"Has {bank} an english website?"
await agent.parse_message_using_nlu_interpreter(q)

{'intent': {'name': 'query_knowledge_base', 'confidence': 0.9951472878456116},
 'entities': [{'start': 4,
   'end': 17,
   'value': 'Wirecard Bank',
   'entity': 'bank',
   'confidence': 0.863771242049505,
   'extractor': 'CRFEntityExtractor'},
  {'start': 21,
   'end': 36,
   'value': 'english website',
   'entity': 'attribute',
   'confidence': 0.9058475601258706,
   'extractor': 'CRFEntityExtractor'}],
 'intent_ranking': [{'name': 'query_knowledge_base',
   'confidence': 0.9951472878456116},
  {'name': 'bot_challenge', 'confidence': 0.0031751112546771765},
  {'name': 'goodbye', 'confidence': 0.0004062456137035042},
  {'name': 'deny', 'confidence': 0.00038728865911252797},
  {'name': 'greet', 'confidence': 0.0003549976972863078},
  {'name': 'affirm', 'confidence': 0.0002805920667015016},
  {'name': 'mood_unhappy', 'confidence': 0.00024087259953375906},
  {'name': 'mood_great', 'confidence': 7.629006177012343e-06}],
 'text': 'Has Wirecard Bank an english website?'}

In [10]:
bank='landwirtschaftliche rentenbank'
q=f"Has {bank} an english website?"
await agent.parse_message_using_nlu_interpreter(q)

{'intent': {'name': 'query_knowledge_base', 'confidence': 0.9981531500816345},
 'entities': [{'start': 4,
   'end': 34,
   'value': 'landwirtschaftliche rentenbank',
   'entity': 'bank',
   'confidence': 0.8485503762832126,
   'extractor': 'CRFEntityExtractor'},
  {'start': 38,
   'end': 53,
   'value': 'english website',
   'entity': 'attribute',
   'confidence': 0.9051008824387026,
   'extractor': 'CRFEntityExtractor'}],
 'intent_ranking': [{'name': 'query_knowledge_base',
   'confidence': 0.9981531500816345},
  {'name': 'bot_challenge', 'confidence': 0.001369673409499228},
  {'name': 'greet', 'confidence': 0.0001675546809565276},
  {'name': 'goodbye', 'confidence': 0.00010590816236799583},
  {'name': 'deny', 'confidence': 8.959946717368439e-05},
  {'name': 'affirm', 'confidence': 6.096474317018874e-05},
  {'name': 'mood_unhappy', 'confidence': 5.244999556452967e-05},
  {'name': 'mood_great', 'confidence': 7.39153620088473e-07}],
 'text': 'Has landwirtschaftliche rentenbank an englis

In [13]:
# restaurant='Berlin Burrito Company'
restaurant='Marubi Ramen'
q=f"What is the price range of {restaurant}?"
await agent.parse_message_using_nlu_interpreter(q)

{'intent': {'name': 'query_knowledge_base', 'confidence': 0.9932721257209778},
 'entities': [{'start': 12,
   'end': 23,
   'value': 'price-range',
   'entity': 'attribute',
   'confidence': 0.956693150086852,
   'extractor': 'CRFEntityExtractor',
   'processors': ['EntitySynonymMapper']},
  {'start': 27,
   'end': 33,
   'value': 'Marubi',
   'entity': 'restaurant',
   'confidence': 0.12171841841973478,
   'extractor': 'CRFEntityExtractor'}],
 'intent_ranking': [{'name': 'query_knowledge_base',
   'confidence': 0.9932721257209778},
  {'name': 'bot_challenge', 'confidence': 0.0031384380999952555},
  {'name': 'greet', 'confidence': 0.00174444739241153},
  {'name': 'mood_unhappy', 'confidence': 0.0005877401563338935},
  {'name': 'goodbye', 'confidence': 0.0005116494139656425},
  {'name': 'affirm', 'confidence': 0.000372273352695629},
  {'name': 'deny', 'confidence': 0.00035593172651715577},
  {'name': 'mood_great', 'confidence': 1.7417069102521054e-05}],
 'text': 'What is the price range